In [3]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

from pyspark.sql.functions import *

### 1. Determine los tres países con mayor número de jugadores(jugadores nacidos en ese país). El resultado debe estar ordenado de forma descendente.

In [4]:
players = spark.read.option('header','true').option('inferSchema', 'true').csv('./data_exc/players.csv')
app = spark.read.option('header','true').option('inferSchema', 'true').csv('./data_exc/appearances.csv')
competition =spark.read.option('header','true').option('inferSchema', 'true').csv('./data_exc/competitions.csv')
games = spark.read.option('header','true').option('inferSchema', 'true').csv('./data_exc/games.csv')

### 2. Obtenga la lista de jugadores con tarjeta roja. La salida debe contener dos columnas, el nombre de pila del jugador y la cantidad de tarjetas rojas que tiene.

In [34]:
players.join(app, ['player_id']).select('player_id', 'pretty_name', 'red_cards').where(col('red_cards') != 0).groupBy('pretty_name').count().show()
#orderBy(count('pretty_name').desc()).

+--------------------+-----+
|         pretty_name|count|
+--------------------+-----+
|Dimitrios Chatzii...|    1|
|  Oleksandr Vasyliev|    1|
|   Kevin Van Diermen|    1|
|         Axel Witsel|    1|
|     Haris Seferovic|    2|
|      Francis Dickoh|    1|
|     Steven Thompson|    1|
|     Pele Van Anholt|    1|
|    Fedor Kudryashov|    3|
|       Armend Dallku|    1|
|          Ivan Ramis|    2|
|   Aleksandar Pantic|    1|
|          Mark Brown|    1|
|       Rick Karsdorp|    2|
|     Sotiris Balafas|    2|
|        Davy Propper|    2|
|         Pierre Webo|    1|
|        Kiko Casilla|    1|
|         Berat Tosun|    1|
| Grzegorz Krychowiak|    3|
+--------------------+-----+
only showing top 20 rows



### 3. ¿Cuántos juegos se jugaron en la Premier League? La salida debe contener dos columnas, el nombre de la liga y la cantidad de juegos que se jugaron en ella.

In [37]:
competition.printSchema()
games.printSchema()

root
 |-- competition_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- type: string (nullable = true)
 |-- country_id: integer (nullable = true)
 |-- country_name: string (nullable = true)
 |-- domestic_league_code: string (nullable = true)
 |-- confederation: string (nullable = true)
 |-- url: string (nullable = true)

root
 |-- game_id: integer (nullable = true)
 |-- competition_code: string (nullable = true)
 |-- season: integer (nullable = true)
 |-- round: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- home_club_id: integer (nullable = true)
 |-- away_club_id: integer (nullable = true)
 |-- home_club_goals: integer (nullable = true)
 |-- away_club_goals: integer (nullable = true)
 |-- home_club_position: integer (nullable = true)
 |-- away_club_position: integer (nullable = true)
 |-- stadium: string (nullable = true)
 |-- attendance: integer (nullable = true)
 |-- referee: string (nullable = true)
 |-- url: string (nullable = true)



In [53]:
competition.join(games, col('competition_id') == col('competition_code')).groupBy('name').count().where(col('name')=='premier-league').show()
# competition.distinct().show()

+--------------+-----+
|          name|count|
+--------------+-----+
|premier-league| 2809|
+--------------+-----+



### 4. Obtenga las tres ligas con mayor número de asistencia de público teniendo en cuenta todos los juegos que se jugaron en ellas. El resultado debe estar ordenado de forma descendente y tener dos columnas, el nombre de la liga y la asistencia total.

In [67]:
# games.groupBy('competition_code').sum('attendance').show()
competition.join(games, col('competition_id') == col('competition_code')).groupBy('name').sum('attendance').orderBy(col('sum(attendance)').desc()).show()  #attendance

+--------------------+---------------+
|                name|sum(attendance)|
+--------------------+---------------+
|      premier-league|       86964852|
|          bundesliga|       78102473|
|              laliga|       62943533|
|             serie-a|       53475147|
|             ligue-1|       51593963|
|uefa-champions-le...|       35154225|
|          eredivisie|       34572418|
|       europa-league|       28710888|
|        premier-liga|       25823581|
|  liga-portugal-bwin|       20072843|
|  jupiler-pro-league|       17817099|
|           super-lig|       17455236|
|scottish-premiership|       17379753|
|europa-league-qua...|       12810167|
|uefa-champions-le...|        9479701|
|             efl-cup|        9162166|
|           dfb-pokal|        8404075|
|         superligaen|        7945555|
|        copa-del-rey|        7052640|
|      super-league-1|        6417136|
+--------------------+---------------+
only showing top 20 rows

